# Lab #4 (v2): Instacart 장바구니 데이터를 이용한 연관규칙 분석

실제 온라인 식료품 마켓인 **Instacart**의 거래 데이터를 **Kaggle API**를 통해 직접 다운로드하여 분석해 보겠습니다. 이 데이터셋은 여러 파일로 나뉘어 있어, 데이터를 병합하고 가공하는 과정이 추가됩니다.

이 과제를 통해 우리는 데이터 전처리부터 연관규칙 생성, 그리고 의미 있는 규칙을 해석하여 비즈니스 인사이트를 도출하는 전 과정을 경험하게 됩니다.

### 과제 목표
1. Kaggle API를 설정하고 Instacart 데이터셋을 다운로드합니다.
2. 여러 CSV 파일(`orders`, `products`, `order_products__prior`)을 로드하고, `merge`를 통해 분석에 필요한 데이터프레임을 생성합니다.
3. `Apriori` 알고리즘을 사용하여 빈번하게 발생하는 상품 조합을 찾습니다.
4. `association_rules`를 통해 연관규칙을 생성하고, **신뢰도(Confidence)**와 **향상도(Lift)**를 기준으로 의미 있는 규칙을 필터링합니다.
5. 특정 상품(예: 'Banana')과 연관성이 높은 상품들을 찾아내고, 시각화를 통해 결과를 분석합니다.

### [준비] Kaggle API 설정

과제를 진행하려면 Kaggle API Key가 필요합니다.
1. [Kaggle](https://www.kaggle.com)에 로그인 후, 'Your Profile' -> 'Account' -> 'API' 섹션으로 이동합니다.
2. 'Create New Token' 버튼을 클릭하면 `kaggle.json` 파일이 다운로드됩니다.
3. ~/.kaggle/kaggle.json 경로에 저장하세요. (또는 username과 key를 확인하여 아래 코드를 실행후 입력하세요.)

In [1]:
import kaggle
import os
import json

In [ ]:
# kaggle.json 파일 경로
kaggle_json_path = os.path.expanduser('~/.kaggle/kaggle.json')
# kaggle.json 파일이 없으면 생성
if not os.path.exists(kaggle_json_path):
    os.makedirs(os.path.dirname(kaggle_json_path), exist_ok=True)
    
    # Kaggle API 인증 정보 입력 받기
    username = input("Kaggle 사용자 이름을 입력하세요: ")
    key = input("Kaggle API 키를 입력하세요: ")
    
    # kaggle.json 파일 생성
    kaggle_json = {
        "username": username,
        "key": key
    }
    
    with open(kaggle_json_path, 'w') as f:
        json.dump(kaggle_json, f)
    
    # 파일 권한 설정 (Kaggle API 요구사항)
    os.chmod(kaggle_json_path, 0o600)
    
    print("Kaggle API 인증이 완료되었습니다.")
else:
    print("이미 Kaggle API 인증이 설정되어 있습니다.")


### [준비] 데이터 다운로드 및 로드

1. Kaggle CLI를 사용하여 `yasserh/instacart-online-grocery-basket-analysis-dataset` 데이터셋을 다운로드하고 압축을 해제하세요.
2. 필요한 3개의 파일(`orders.csv`, `products.csv`, `order_products__prior.csv`)을 pandas DataFrame으로 로드하세요.

In [2]:
import pandas as pd
import numpy as np
from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import apriori, association_rules
import plotly.express as px

# Kaggle 데이터셋 다운로드 및 압축 해제
#(도커/로컬 환경에 따라 저장위치를 바꾸세요)
base_path = "../datasets/ml/instacart/"

In [ ]:
# ggle API를 사용하여 데이터셋 다운로드 
kaggle.api.dataset_download_files('yasserh/instacart-online-grocery-basket-analysis-dataset', path=base_path, unzip=True)

In [ ]:
# 데이터 로드
orders_df = pd.read_csv(os.path.join(base_path,'orders.csv'))
products_df = pd.read_csv(os.path.join(base_path,'products.csv'))
order_products_prior_df = pd.read_csv(os.path.join(base_path,'order_products__prior.csv'))

print('--- orders_df ---')
print(orders_df.head())
print('\n--- products_df ---')
print(products_df.head())
print('\n--- order_products_prior_df ---')
print(order_products_prior_df.head())

### [문제 1] 데이터 병합 및 샘플링

전체 데이터는 매우 크므로, 분석을 위해 데이터의 일부만 사용하겠습니다.
1. `orders_df`에서 `eval_set`이 'prior'인 주문 중 20,000개를 무작위로 샘플링하여 `sampled_orders_df`를 만듭니다. (`random_state=42`로 설정하여 재현성을 확보하세요.)
2. `order_products_prior_df`와 `sampled_orders_df`를 `order_id` 기준으로 병합(merge)하여 샘플링된 주문에 대한 상품 목록을 얻습니다.
3. 위 결과와 `products_df`를 `product_id` 기준으로 병합하여 `product_name`을 추가한 최종 분석용 데이터프레임 `df_merged`를 만드세요.

In [ ]:
# 1. 20,000개 주문 샘플링
# prior_orders_df = orders_df[orders_df['eval_set'] == 'prior']
# sampled_orders_df = prior_orders_df.sample(n=20000, random_state=42)

# 2. 주문-상품 데이터 병합
# df_order_products = pd.merge(sampled_orders_df, order_products_prior_df, on='order_id', how='inner')

# 3. 상품 이름 데이터 병합
# df_merged = pd.merge(df_order_products, products_df, on='product_id', how='inner')

# 최종 데이터 확인
# print(df_merged.shape)
# df_merged[['order_id', 'product_id', 'product_name']].head()

### [문제 2] 트랜잭션 데이터로 변환

연관규칙 분석을 위해, `df_merged`를 각 `order_id`별로 구매한 상품 목록(`product_name`)의 리스트 형태로 변환한 후, `TransactionEncoder`를 사용하여 One-Hot Encoded DataFrame을 생성하세요.

In [ ]:
# order_id를 기준으로 product_name을 리스트로 묶기
# transactions = df_merged.groupby('?')['?'].apply(?).tolist()

# TransactionEncoder를 사용하여 One-Hot 인코딩
# te = TransactionEncoder()
# te_ary = te.fit(transactions).transform(transactions)
# df_encoded = pd.DataFrame(te_ary, columns=te.columns_)

# df_encoded.head()

### [문제 3] Apriori 알고리즘 적용 및 연관규칙 생성

1. `apriori` 알고리즘을 실행하여 **최소 지지도(min_support)가 0.01 이상**인 빈번 아이템셋을 찾으세요.
2. `association_rules` 함수를 사용하여 **최소 향상도(min_threshold)가 1.5 이상**인 연관규칙을 생성하세요. (`metric='lift'`)

In [ ]:
# Apriori 알고리즘 적용
# frequent_itemsets = apriori(?, min_support=?, use_colnames=True)

# 연관규칙 생성
# rules = association_rules(?, metric=?, min_threshold=?)

# 신뢰도(confidence)와 향상도(lift) 기준으로 정렬하여 상위 10개 규칙 확인
# rules.sort_values(by=['lift', 'confidence'], ascending=False).head(10)

### [문제 4] 특정 상품 관련 규칙 분석

인기 상품인 **'Banana'**가 구매되었을 때, 어떤 상품들이 함께 구매되는 경향이 있는지 분석해 보세요.
1. 생성된 `rules`의 `antecedents`(조건절)에 'Banana'가 포함된 규칙들을 필터링하세요.
2. 필터링된 규칙들을 `lift` 기준으로 내림차순 정렬하여 확인하세요.

In [ ]:
# 조건절에 'Banana'가 포함된 규칙 찾기
# is_banana_antecedent = rules['antecedents'].apply(lambda x: 'Banana' in x)
# banana_rules = rules[?]

# print("'Banana'와 함께 구매되는 상품 Top 10 (by Lift)")
# banana_rules.sort_values(by='lift', ascending=False).head(10)

### [문제 5] 분석 결과 시각화

**문제 4**에서 생성한 전체 `rules` 데이터프레임을 사용하여 지지도, 신뢰도, 향상도의 관계를 시각화하세요.
- `x`축은 `support`, `y`축은 `confidence`, `color`는 `lift`로 설정한 Scatter Plot을 `Plotly Express`로 그리세요.

In [ ]:
# 시각화를 위해 frozenset을 문자열로 변환
# rules['antecedents_str'] = rules['antecedents'].apply(lambda x: ', '.join(list(x)))
# rules['consequents_str'] = rules['consequents'].apply(lambda x: ', '.join(list(x)))

# Scatter plot 생성
# import plotly.express as px
# fig = px.scatter(?, 
#                  x=?, 
#                  y=?, 
#                  color=?, 
#                  hover_data=['antecedents_str', 'consequents_str'],
#                  title="Instacart Association Rules (Sampled)")
# fig.show()